# Time Series Analysis with Twitter and CDC data

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Load required libraries
import pandas as pd
from pandas import Series
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
#http://www.cs.uml.edu/~bliu/pub/CPNS_2011.pdf
df = pd.read_csv('FluViewPhase2Data/ILINet.csv')
df.head()

PERCENTAGE OF VISITS FOR INFLUENZA-LIKE-ILLNESS REPORTED BY SENTINEL PROVIDERS
REGION TYPE REGION YEAR WEEK % WEIGHTED ILI %UNWEIGHTED ILI AGE 0-4 AGE 25-49 AGE 25-64 AGE 5-24 AGE 50-64 AGE 65 ILITOTAL NUM. OF PROVIDERS                                     TOTAL PATIENTS                            
National    X      2017 40   1.26686        1.27887         3852    2502      X         4567     1075      877    12873    2227                                                         1006594                            
                        41   1.30548        1.32949         4119    2686      X         4463     1078      952    13298    2252                                                         1000234                            
                        42   1.3427         1.40743         4416    2823      X         5007     1142      976    14364    2273                                                         1020587                            
                        43   1.51794        1.56987         5012    2927      X         5539     1327      1055   15860    2272                                                         1010276

In [ ]:
#for 2017 - week 40 is week of October 2, 2017
#need to convert weeks, keep total patients for ILI illness

In [ ]:
#will need to delete retweets, tweets from the same user within a week (not a new case)
#normalize scale of CDC visits and tweets 

## Visualizations

In [1]:
#include framework model here -- tracking influenza-like illness 

In [2]:
#show distribution (count of flu-related tweets) over time 
#https://github.com/learn-co-students/ds-ts-visualize_ts-nyc-career-ds-062518/tree/solution
#need Pandas Series with date and tweet count per date 
#overlay with CDC data to see how it corresponds -- have dropdown menu 
series_name.plot(figsize = (20,6))
plt.show()

In [3]:
#zoom in to see time distribution by state 

In [ ]:
#maybe heat map by month and days 
plt.matshow(months_df, interpolation=None, aspect='auto', cmap=plt.cm.Spectral_r)
plt.show()

## Correlation between CDC reports and ILI-tweets

### Auto-Correlation 

In [ ]:
#check auto-correlation to assess seasonality? (maybe)

### Pearson Correlation Coefficient

In [8]:
#pearson correlation coefficient between CDC ILI visits and ILI tweets 

## ARIMA Modeling to Forecast Influenza-Like Cases

In [ ]:
#predict ILI incidence as a linear function of current and 
#old Social Network data and historical ILI data (CDC data) -- assess model goodness of fit with AIC, lowest value 
#https://github.com/learn-co-students/ds-ts-ARIMA-nyc-career-ds-062518/tree/solution

### Model with CDC data only

### Model with CDC & Twitter data

### Model Selection: Overview of Goodness-of-Fit Measures

### Model Validation: Forecasting

In [9]:
#one-step ahead forecasting for best fitting model - comparing predicted values to real values of the time series, 
#which will help us understand the accuracy of our forecasts (use cross-fold validation)

## Producing & Visualizing Forecasts